# Futur atlas 2020-2024

In [ ]:
import sys
sys.version

In [ ]:
import os
import re
import pandas as pd
import numpy as np
#from lxml import etree
import simplekml as skml # Simple KML generation API
import kmlcircle # Circle generation for KML generation
#import requests
#import folium
from math import sqrt

# Génération KML pour carrés 1km

In [ ]:
squareCenters = \
[
    (46.108257, 3.071173), # Les Plagnots, Ebreuil (centre équestre)
]

In [ ]:
squareWidth = 1000 # m
squareRadius = sqrt(2*(squareWidth/2)**2)

In [ ]:
kml = skml.Kml()

In [ ]:
lineStyle = skml.LineStyle(color=skml.Color.red, width=1)

In [ ]:
for lat, long in squareCenters:
    ls = kml.newlinestring(coords=kmlcircle.spoints(long=long, lat=lat, meters=squareRadius, n=4, offset=45))
    ls.linestyle = lineStyle

In [ ]:
kml.save('carres-1km.kml')